# Simple implementation of reactive patterns in Python

In [1]:
from typing import Callable, List, TypeVar, Generic, Optional, Any, Iterable
import threading
from time import sleep

## Base classes

### Some needed types

In [2]:
T = TypeVar("T")
U = TypeVar("U")
V = TypeVar("V")

NextCallback = Optional[Callable[[T], None]]
ErrorCallback = Optional[Callable[[Exception], None]]
CompleteCallback = Optional[Callable[[], None]]

### Observer

In [3]:
class Observer(Generic[T]):

    def __init__(
            self,
            next: NextCallback = None,
            error: ErrorCallback = None,
            complete: CompleteCallback = None):
        super().__init__()
        self._next = next
        self._error = error
        self._complete = complete
    
    def next(self, value: T) -> None:
        if self._next is not None:
            self._next(value)

    def error(self, error: Exception) -> None:
        if self._error is not None:
            self._error(error)

    def complete(self) -> None:
        if self._complete is not None:
            self._complete()

### Subscription

In [4]:
class Subscription:

    def __init__(self, dispose: Callable[[], None]):
        self._dispose = dispose

    def dispose(self) -> None:
        self._dispose()

### Observable

In [5]:
class Observable(Generic[T]):

    def __init__(self, subscribe: Callable[[Observer[T]], Subscription]):
        super().__init__()
        self._subscribe = subscribe

    def subscribe(
            self,
            next: NextCallback = None,
            error: ErrorCallback = None,
            complete: CompleteCallback = None) -> Subscription:
        
        return self._subscribe(
            Observer(next=next, error=error, complete=complete)
        )
    
    def take(self, n: int) -> 'Observable[T]': # type: ignore
        pass
    
    def map(self, func: Callable[[T], U]) -> 'Observable[U]': # type: ignore
        pass

    def filter(self, test: Callable[[T], bool]) -> 'Observable[T]': # type: ignore
        pass

    def do(
        self,
        next: NextCallback = None,
        error: ErrorCallback = None,
        complete: CompleteCallback = None) -> 'Observable[T]': # type: ignore
        pass

### Subject

In [6]:
class Subject(Generic[T]):

    def __init__(self):
        super().__init__()
        self._observers: List[Observer[T]] = []

    def next(self, value: T) -> None:
        for observer in self._observers:
            observer.next(value=value)

    def error(self, error: Exception) -> None:
        for observer in self._observers:
            observer.error(error=error)

    def complete(self) -> None:
        for observer in self._observers:
            observer.complete()

    def asObservable(self) -> 'Observable[T]':

        def _subscribe(observer: Observer[T]) -> Subscription:
            return self.subscribe(
                next=observer.next,
                error=observer.error,
                complete=observer.complete
            )

        return Observable(subscribe=_subscribe)
    
    def subscribe(
            self,
            next: NextCallback = None,
            error: ErrorCallback = None,
            complete: CompleteCallback = None) -> Subscription:
        
        observer = Observer(next=next, error=error, complete=complete)
        self._observers.append(observer)

        def _remove() -> None:
            if observer in self._observers:
                self._observers.remove(observer)
        
        return Subscription(dispose=_remove)

## Operators

### Take

Takes n first values then disposes the source observable

0-1-2-3-4-5--->
 
  take(3)
 
0-1-2-|------->

In [7]:
def __take(self: Observable[T], n: int) -> Observable[T]:

    source = self
    
    def _subscribe(observer: Observer[T]) -> Subscription:
        
        def _next(value: T) -> None:
            nonlocal i
            
            if i < n:
                observer.next(value=value)
                i += 1

                if i == n:
                    #observer.complete()
                    subscription.dispose()

        i = 0

        subscription = source.subscribe(
            next=_next,
            error=observer.error,
            complete=observer.complete
        )

        return Subscription(dispose=subscription.dispose)

    return Observable(subscribe=_subscribe)

Observable.take = __take

### Map

Maps each resulting value from stream

0-1-2-3-4-5--->
 
map(x -> x * 2)
 
0-2-4-6-8-10--->

In [8]:
def __map(self: Observable[T], func: Callable[[T], U]) -> Observable[U]:

    source = self

    def _subscribe(observer: Observer[U]) -> Subscription:
        
        def _next(value: T) -> None:
            observer.next(value=func(value))

        subscription = source.subscribe(
            next=_next,
            error=observer.error,
            complete=observer.complete
        )

        return Subscription(dispose=subscription.dispose)

    return Observable(subscribe=_subscribe)

Observable.map = __map

### Filter

Filters value from source stream

-5-9-2-7-3-4---->

filter(x -> x > 4)
 
-5-9---7-------->

In [9]:
def __filter(self: Observable[T], test: Callable[[T], bool]) -> Observable[T]:

    source = self

    def _subscribe(observer: Observer[T]) -> Subscription:
        
        def _next(value: T) -> None:
            if test(value):
                observer.next(value=value)

        subscription = source.subscribe(
            next=_next,
            error=observer.error,
            complete=observer.complete
        )

        return Subscription(dispose=subscription.dispose)

    return Observable(subscribe=_subscribe)

Observable.filter = __filter

### Do

Do something on each value from stream without modifying the stream

0-1-2-3-4-5--->

do(x -> console.log(x))

0-1-2-3-4-5--->

In [10]:
def __do(
        self: Observable[T],
        next: NextCallback = None,
        error: ErrorCallback = None,
        complete: CompleteCallback = None) -> Observable[T]:
    
    source = self

    to_do = Observer(
        next=next,
        error=error,
        complete=complete
    )

    def _subscribe(observer: Observer[T]) -> Subscription:
        
        def _next(value: T) -> None:
            to_do.next(value=value)
            observer.next(value=value)

        def _error(error: Exception) -> None:
            to_do.error(error=error)
            observer.error(error=error)

        def _complete() -> None:
            to_do.complete()
            observer.complete()

        subscription = source.subscribe(
            next=_next,
            error=_error,
            complete=_complete
        )

        return Subscription(dispose=subscription.dispose)

    return Observable(subscribe=_subscribe)

Observable.do = __do

## Generators

### Empty

Returns an observable that completes imediately (empty stream)

-|---------------->

In [11]:
def empty() -> Observable[Any]:

    def _subscribe(observer: Observer[Any]) -> Subscription:
        
        def _dispose() -> None:
            nonlocal is_disposed
            is_disposed = True

        is_disposed = False

        try:
            observer.complete()
        except Exception as e:
            observer.error(e)

        return Subscription(dispose=_dispose)

    return Observable(subscribe=_subscribe)

### Of

Creates an one valued stream

of(5)

-5-|------------------>

In [12]:
def of(value: T) -> Observable[T]:

    def _subscribe(observer: Observer[T]) -> Subscription:
        
        def _dispose() -> None:
            nonlocal is_disposed
            is_disposed = True

        is_disposed = False

        try:
            observer.next(value=value)
            observer.complete()
        except Exception as e:
            observer.error(e)

        return Subscription(dispose=_dispose)

    return Observable(subscribe=_subscribe)

### From Iterable

Creates a stream from an iterable

from_iterable([3,8,5,1])

-3-8-5-1-|-------->

In [13]:
def from_iterable(iterable: Iterable[T]) -> Observable[T]:
    
    def _subscribe(observer: Observer[T]) -> Subscription:
        
        def _dispose() -> None:
            nonlocal is_disposed
            is_disposed = True
        
        def run() -> None:
            try:
                for value in iterable:
                    observer.next(value=value)
                observer.complete()
            except Exception as e:
                observer.error(e)

        is_disposed = False

        threading.Thread(target=run).start()

        return Subscription(dispose=_dispose)

    return Observable(subscribe=_subscribe)

### From Range

Creates a stream from a range

range(4,8)

-4-5-6-7-8-|--->

In [14]:
def from_range(start: int, stop: int, step: int = 1) -> Observable[int]:
    return from_iterable(range(start, stop, step))

### Creates a stream from interval

-0-1-2-3-4-5-6----->

In [15]:
def from_interval(period: int) -> Observable[int]:
    
    def _subscribe(observer: Observer[int]) -> Subscription:
        
        def _dispose() -> None:
            nonlocal is_disposed
            is_disposed = True
        
        def run() -> None:
            nonlocal i

            try:
                while is_disposed == False:
                    observer.next(value=i)
                    i += 1
                    sleep(period)
                observer.complete()
            except Exception as e:
                observer.error(e)

        i = 0
        is_disposed = False

        threading.Thread(target=run).start()

        return Subscription(dispose=_dispose)

    return Observable(subscribe=_subscribe)

## Examples

### Empty stream

In [16]:
empty().subscribe(
    next=lambda o: print(o),
    complete=lambda: print('completed')
)

completed


### Unique valued stream

In [17]:
of('coucou').subscribe(
    next=lambda o: print(o),
    complete=lambda: print('completed')
)

coucou
completed


### Map values

In [18]:
of(5).map(func=lambda x: 2*x).subscribe(
    next=lambda o: print(o),
    complete=lambda: print('completed')
)

10
completed


### Takes from range

In [19]:
from_range(1, 10).take(5).subscribe(
    next=lambda o: print(o),
    complete=lambda: print('completed')
)


1
2
3
4
5
completed


### Filter from range

In [20]:
from_range(1, 10).filter(lambda o: o%2==0).subscribe(
    next=lambda o: print(o),
    complete=lambda: print('completed')
)

2
4
6
8
completed


### Takes from interval

In [21]:
from_interval(period=1).take(3).map(lambda o: o+1).subscribe(
    next=lambda o: print(o),
    complete=lambda: print('completed')
)

sleep(3)

1
2
3


completed
